In [1]:
import ultralytics
import os
import torch
from pathlib import Path
import shutil
import random
import gc
import json

In [2]:
torch.cuda.is_available()

True

# Код

In [31]:
def create_processed_info_for_directory(path):
    label_dirs = os.listdir(path)

    processed_info = dict()

    for label in label_dirs:
        processed_info[label] = set()

        label_path = os.path.join(path, label)
        images = os.listdir(label_path)

        for image in images:
            processed_info[label].add('_'.join(image.split('.')[0].split('_')[:-1]))
    
    for label in label_dirs:
        processed_info[label] = list(processed_info[label])

    with open(PROCESSED_INFO_PATH, 'w') as f:
        json.dump(processed_info, f, indent=4)

# Прогоняем датасет через детекцию людей

In [3]:
CURRENT_FOLDER = r"/mnt/c/Notebooks/Workout"
DATASET_PATH = r"/mnt/c/Users/dlyko/.cache/kagglehub/datasets/hasyimabdillah/workoutfitness-video/versions/5"
PROCESSED_INFO_PATH = os.path.join(CURRENT_FOLDER, 'processed.json')

EXERCISES = ['leg raises', 'pull Up', 'push-up', 'squat', 'tricep dips']

In [4]:
model = ultralytics.YOLO("yolo11s.pt")

In [ ]:
processed_info = json.load(PROCESSED_INFO_PATH)

for label in processed_info.keys():
    processed_info[label] = set(processed_info[label])

for class_dir in EXERCISES:
    full_dir_path = os.path.join(DATASET_PATH, class_dir)
    video_paths = os.listdir(full_dir_path)
    random.shuffle(video_paths)

    if class_dir not in processed_info:
        processed_info[class_dir] = set()

    for video in video_paths:
        if video in processed_info[class_dir]:
            print(video + " has been alrady processed. Skipped")
            continue

        video_path = os.path.join(full_dir_path, video)
        save_path = os.path.join(CURRENT_FOLDER, 'runs/human_detect/full', class_dir)

        results = model(video_path, classes=[0], save_crop=True, project=save_path, name='predict')
        for crop in os.listdir(os.path.join(save_path, 'predict', 'crops', 'person')):
            shutil.move(os.path.join(save_path, 'predict', 'crops', 'person', crop), os.path.join(save_path, crop))
        shutil.rmtree(os.path.join(save_path, 'predict'))

        processed_info[class_dir].add(video)
        with open(PROCESSED_INFO_PATH, 'w') as f:
            json.dump(processed_info, f)

        gc.collect()
        torch.cuda.empty_cache()

In [33]:
create_processed_info_for_directory(os.path.join(CURRENT_FOLDER, 'runs/human_detect/full'))

# Прогоняем датасет через оценку позы

In [5]:
model = ultralytics.YOLO("yolo11s-pose.pt")

In [7]:
for class_dir in EXERCISES:
    full_dir_path = os.path.join(DATASET_PATH, class_dir)
    video_paths = os.listdir(full_dir_path)
    random.shuffle(video_paths)
    for video in video_paths:
        video_path = os.path.join(full_dir_path, video)
        save_path = os.path.join(CURRENT_FOLDER, 'runs/pose/full', class_dir)
        
        model(video_path, classes=[0], save=True, project=save_path, name='predict')
        new_video = video.split('.')[0] + ".avi"
        shutil.move(os.path.join(save_path, 'predict', new_video), os.path.join(save_path, new_video))
        os.rmdir(os.path.join(save_path, 'predict'))

        gc.collect()
        torch.cuda.empty_cache()

# Эксперимент 1

Обучим классификатор YOLO на 4 классах упражнений

Датасет с Kaggle, для каждого видео была вырезана рамка с человеком. То есть в каждом кадре только человек

Один человек не повторяется в train, val и test (но это не точно, разделение было ручное)

In [3]:
model = ultralytics.YOLO("./yolo11s-cls.pt")

In [4]:
results = model.train(data="/mnt/c/Notebooks/Workout/four_classes/steps_divided", 
                      project='/mnt/c/Notebooks/Workout/runs',
                      epochs=12, imgsz=320)

New https://pypi.org/project/ultralytics/8.3.206 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.202 🚀 Python-3.11.13 torch-2.8.0+cu128 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/mnt/c/Notebooks/Workout/four_classes/steps_divided, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=12, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=./yolo11s-cls.pt, momentum=0.937, mosaic=1.0, multi_

## Проверим на тестовых данных

In [6]:
TESTING_IMAGES = '/mnt/c/Notebooks/Workout/four_classes/steps_divided/test'
WEIGHTS = r'/mnt/c/Notebooks/Workout/runs/train7/weights/best.pt'
SAVE_TEST_TO = '/mnt/c/Notebooks/Workout/runs/train7/testing'

# os.mkdir(SAVE_TEST_TO)

model = ultralytics.YOLO(WEIGHTS)

In [8]:
for label in [l for l in os.listdir(TESTING_IMAGES) if os.path.isdir(os.path.join(TESTING_IMAGES, l))]:
    label_dir = os.path.join(TESTING_IMAGES, label)
    images = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if not os.path.isdir(os.path.join(label_dir, f))]

    if label == 'no exercise':
        images = images[::4]
    
    results = model(images)
    for result in results:
        image_name = result.path.split('/')[-1]
        result.save(filename=os.path.join(SAVE_TEST_TO, image_name))
    
    gc.collect()
    torch.cuda.empty_cache()


0: 320x320 barbell biceps curl up 1.00, no exercise 0.00, leg raises up 0.00, push-up down 0.00, pull Up up 0.00, 4.4ms
1: 320x320 barbell biceps curl up 1.00, no exercise 0.00, leg raises up 0.00, push-up down 0.00, pull Up up 0.00, 4.4ms
2: 320x320 barbell biceps curl up 1.00, no exercise 0.00, pull Up up 0.00, leg raises up 0.00, push-up down 0.00, 4.4ms
3: 320x320 barbell biceps curl up 1.00, no exercise 0.00, pull Up up 0.00, leg raises up 0.00, push-up down 0.00, 4.4ms
4: 320x320 barbell biceps curl up 1.00, no exercise 0.00, pull Up up 0.00, leg raises up 0.00, push-up down 0.00, 4.4ms
5: 320x320 barbell biceps curl up 1.00, no exercise 0.00, pull Up up 0.00, leg raises up 0.00, push-up down 0.00, 4.4ms
6: 320x320 barbell biceps curl up 1.00, no exercise 0.00, pull Up up 0.00, push-up down 0.00, leg raises up 0.00, 4.4ms
7: 320x320 barbell biceps curl up 1.00, no exercise 0.00, pull Up up 0.00, leg raises up 0.00, push-up down 0.00, 4.4ms
8: 320x320 barbell biceps curl up 1.00,

In [5]:
for image in [os.path.join(TESTING_IMAGES, f) for f in os.listdir(TESTING_IMAGES) if not os.path.isdir(os.path.join(label_dir, f))]:
    results = model(image)
    for result in results:
        image_name = result.path.split('/')[-1]
        result.save(filename=os.path.join(SAVE_TEST_TO, 'no label', image_name))
    
    gc.collect()
    torch.cuda.empty_cache()


image 1/1 /mnt/c/Notebooks/Workout/four_classes/steps_divided/test/barbell biceps curl_1_25.jpg: 320x320 barbell biceps curl up 1.00, pull Up up 0.00, leg raises up 0.00, push-up down 0.00, 100.5ms
Speed: 115.1ms preprocess, 100.5ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 /mnt/c/Notebooks/Workout/four_classes/steps_divided/test/barbell biceps curl_1_26.jpg: 320x320 barbell biceps curl up 1.00, leg raises up 0.00, pull Up up 0.00, push-up down 0.00, 8.7ms
Speed: 33.4ms preprocess, 8.7ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 /mnt/c/Notebooks/Workout/four_classes/steps_divided/test/barbell biceps curl_1_27.jpg: 320x320 barbell biceps curl up 1.00, leg raises up 0.00, pull Up up 0.00, push-up down 0.00, 8.0ms
Speed: 6.3ms preprocess, 8.0ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 /mnt/c/Notebooks/Workout/four_classes/steps_divided/test/barbell biceps curl_1_28.jpg: 320x320 barbell 

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f35847ff850>>
Traceback (most recent call last):
  File "/home/dlyko/miniconda3/envs/workout/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 781, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 



image 1/1 /mnt/c/Notebooks/Workout/four_classes/steps_divided/test/leg raises_9_91.jpg: 320x320 pull Up up 0.60, leg raises up 0.40, push-up down 0.00, barbell biceps curl up 0.00, 23.2ms
Speed: 7.0ms preprocess, 23.2ms inference, 0.1ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 /mnt/c/Notebooks/Workout/four_classes/steps_divided/test/leg raises_9_92.jpg: 320x320 pull Up up 0.58, leg raises up 0.42, push-up down 0.00, barbell biceps curl up 0.00, 16.6ms
Speed: 5.9ms preprocess, 16.6ms inference, 0.1ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 /mnt/c/Notebooks/Workout/four_classes/steps_divided/test/leg raises_9_93.jpg: 320x320 pull Up up 0.53, leg raises up 0.47, push-up down 0.00, barbell biceps curl up 0.00, 26.7ms
Speed: 5.5ms preprocess, 26.7ms inference, 0.1ms postprocess per image at shape (1, 3, 320, 320)

image 1/1 /mnt/c/Notebooks/Workout/four_classes/steps_divided/test/leg raises_9_94.jpg: 320x320 pull Up up 0.52, leg raises up 0.48, push-up 

KeyboardInterrupt: 

## Как работает, если упражнения нет

In [9]:
TESTING_IMAGES = '/mnt/c/Notebooks/Workout/runs/train4/non_dataset_images'
SAVE_TEST_TO = os.path.join(TESTING_IMAGES, 'results')

model = ultralytics.YOLO(WEIGHTS)

In [12]:
for image in os.listdir(TESTING_IMAGES):
    image_path = os.path.join(TESTING_IMAGES, image)
    result = model(image_path)[0]

    print(result.names)
    print(result.path, result.probs.top1)
    result.save(filename=os.path.join(SAVE_TEST_TO, image))
    
    gc.collect()
    torch.cuda.empty_cache()


image 1/1 /mnt/c/Notebooks/Workout/runs/train4/non_dataset_images/b57ad2e204f452f0d3327e806e69c40d.png: 320x320 pull Up up 0.90, leg raises up 0.09, push-up down 0.00, barbell biceps curl up 0.00, 14.6ms
Speed: 54.6ms preprocess, 14.6ms inference, 0.1ms postprocess per image at shape (1, 3, 320, 320)
{0: 'barbell biceps curl up', 1: 'leg raises up', 2: 'pull Up up', 3: 'push-up down'}
/mnt/c/Notebooks/Workout/runs/train4/non_dataset_images/b57ad2e204f452f0d3327e806e69c40d.png 2

image 1/1 /mnt/c/Notebooks/Workout/runs/train4/non_dataset_images/i (1).jpg: 320x320 pull Up up 1.00, push-up down 0.00, leg raises up 0.00, barbell biceps curl up 0.00, 14.3ms
Speed: 29.2ms preprocess, 14.3ms inference, 0.1ms postprocess per image at shape (1, 3, 320, 320)
{0: 'barbell biceps curl up', 1: 'leg raises up', 2: 'pull Up up', 3: 'push-up down'}
/mnt/c/Notebooks/Workout/runs/train4/non_dataset_images/i (1).jpg 2

image 1/1 /mnt/c/Notebooks/Workout/runs/train4/non_dataset_images/i (2).jpg: 320x320 

error: OpenCV(4.12.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:1051: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'imwrite_'


# Эксперимент 2

**Модель:** yolo11s-cls.pt

**Датасет:** 
- Датасет с Kaggle, для каждого видео была вырезана рамка с человеком. В каждый класс входит только *окончание* упражнения
- Изображения без упражнений: прочие позы людей и *начала* упражнений
    
**Разделение:** Один человек не повторяется в train, val и test (разделение было ручное)

In [3]:
EXERCISE_ENDS = r'/mnt/c/Notebooks/Workout/four_classes/steps_divided'
NO_EXERCISES = r'/mnt/c/Notebooks/Workout/four_classes/no excersises'
FULL_IMAGES = r'/mnt/c/Notebooks/Workout/four_classes/reviewed'

In [14]:
ends_filenames = []
for dir in [d for d in os.listdir(EXERCISE_ENDS) if os.path.isdir(os.path.join(EXERCISE_ENDS, d))]:
    for label in [d for d in os.listdir(os.path.join(EXERCISE_ENDS, dir)) if os.path.isdir(os.path.join(EXERCISE_ENDS, dir, d))]:
        ends_filenames += [os.path.join(dir, label, f) for f in os.listdir(os.path.join(EXERCISE_ENDS, dir, label))]

In [15]:
ends_filenames = set(ends_filenames)

In [20]:
ends_filenames = {v.split('/')[-1]:v for v in ends_filenames}

In [21]:
ends_filenames

{'push-up_11_138.jpg': 'train/push-up down/push-up_11_138.jpg',
 'push-up_51_242.jpg': 'train/push-up down/push-up_51_242.jpg',
 'leg raises_8_205.jpg': 'train/leg raises up/leg raises_8_205.jpg',
 'pull up_1_146.jpg': 'train/pull Up up/pull up_1_146.jpg',
 'push-up_52_96.jpg': 'train/push-up down/push-up_52_96.jpg',
 'pull up_10_166.jpg': 'train/pull Up up/pull up_10_166.jpg',
 'leg raises_7_26.jpg': 'train/leg raises up/leg raises_7_26.jpg',
 'leg raises_6_190.jpg': 'val/leg raises up/leg raises_6_190.jpg',
 'leg raises_19_127.jpg': 'train/leg raises up/leg raises_19_127.jpg',
 'push-up_36_149.jpg': 'val/push-up down/push-up_36_149.jpg',
 'leg raises_19_31.jpg': 'train/leg raises up/leg raises_19_31.jpg',
 'barbell biceps curl_45_361.jpg': 'train/barbell biceps curl up/barbell biceps curl_45_361.jpg',
 'leg raises_10_31.jpg': 'train/leg raises up/leg raises_10_31.jpg',
 'push-up_53_647.jpg': 'train/push-up down/push-up_53_647.jpg',
 'push-up_36_96.jpg': 'val/push-up down/push-up_36_9

In [17]:
full_filenames = []
for label in [d for d in os.listdir(FULL_IMAGES) if os.path.isdir(os.path.join(FULL_IMAGES, d))]:
    full_filenames += [os.path.join(label, f) for f in os.listdir(os.path.join(FULL_IMAGES, label))]

In [18]:
full_filenames = set(full_filenames)

In [22]:
full_filenames = {v.split('/')[-1]:v for v in full_filenames}

In [23]:
full_filenames

{'leg raises_16_31.jpg': 'leg raises/leg raises_16_31.jpg',
 'push-up_17_61.jpg': 'push-up/push-up_17_61.jpg',
 'leg raises_1_60.jpg': 'leg raises/leg raises_1_60.jpg',
 'push-up_35_943.jpg': 'push-up/push-up_35_943.jpg',
 'push-up_53_242.jpg': 'push-up/push-up_53_242.jpg',
 'push-up_36_46.jpg': 'push-up/push-up_36_46.jpg',
 'leg raises_1_281.jpg': 'leg raises/leg raises_1_281.jpg',
 'push-up_42_149.jpg': 'push-up/push-up_42_149.jpg',
 'pull up_16_22.jpg': 'pull Up/pull up_16_22.jpg',
 'barbell biceps curl_3_45.jpg': 'barbell biceps curl/barbell biceps curl_3_45.jpg',
 'pull up_15_65.jpg': 'pull Up/pull up_15_65.jpg',
 'pull up_7_237.jpg': 'pull Up/pull up_7_237.jpg',
 'leg raises_3_2742.jpg': 'leg raises/leg raises_3_2742.jpg',
 'push-up_39_79.jpg': 'push-up/push-up_39_79.jpg',
 'pull up_13_124.jpg': 'pull Up/pull up_13_124.jpg',
 'leg raises_15_100.jpg': 'leg raises/leg raises_15_100.jpg',
 'leg raises_17_163.jpg': 'leg raises/leg raises_17_163.jpg',
 'push-up_19_141.jpg': 'push-up/p

In [25]:
len(ends_filenames.keys()), len(full_filenames.keys())

(8207, 26008)

In [26]:
starts_filenames = set(full_filenames.keys()) - set(ends_filenames.keys())

In [27]:
len(starts_filenames)

17801

In [30]:
for filename in starts_filenames:
    source = os.path.join(FULL_IMAGES, full_filenames[filename])
    dest = os.path.join(NO_EXERCISES, full_filenames[filename].split('/')[-1])
    shutil.move(source, dest)